#### Generate STR input files (variant_catalog.json and .bed file) for EPI2ME workflow
This script is necessary because: 
(1) we want to keep our gene (STR) list up-to-date as new pathogenic variants are identified, 
(2) STRchive and straglr have some differences in the formats of their variant_catalogs and .bed files that contain the list of loci to report. 

This script will pull the latest versions from STRchive and make the variant_catalog and list of loci for straglr/EPI2ME workflow. 
It will also make a .bed file formatted for NanoRepeat.

You can filter your locus list to include only STRs with a certain minimum age of onset. For example, set age_onset_max to 1 if you want to only look at loci with pathogenic effects within the first year of life.

Elizabeth A. Ostrowski
Last updated: 16 August 2024


Load libraries

In [1]:
import re
import json
import requests
import pandas as pd

Specify the output filenames (bed, json)


In [3]:
# Parameters: output filenames (bed, json)
out_straglr_bed_fname = "strs_for_straglr.bed"
out_straglr_json_fname = "updated_variant_catalog_hg38.json"
out_nano_bed_fname = "strs_for_nano.bed"
age_onset_max = 100 # Use this parameter to filter out late-onset diseases 
                    # (e.g., set to 1 if you want diseases with onset within the first year of life.)

In [6]:
# Create and open output files
out_straglr_bed = open(out_straglr_bed_fname, "w")
out_nano_bed = open(out_nano_bed_fname, "w")
out_straglr_json = open(out_straglr_json_fname, "w")

# Get json file from STRchive
json_url = "https://raw.githubusercontent.com/dashnowlab/STRchive/main/data/STRchive-database.json"
resp = requests.get(json_url)
my_json = json.loads(resp.text)

# Reformat the STRchive json to match the straglr format
variant_catalog = []
gene_dict = {}

for record in my_json: # For genes with >1 STR, add "_1", "_2", etc for each one
        gene = record.get("gene")
        
        if gene not in gene_dict:
            gene_dict[gene]=1
        else:
            gene_dict[gene] = gene_dict[gene]+1
            gene = gene + "_" + str(gene_dict[gene])    
        
        #gene = record.get('id')
        normal_max = record.get('normal_max')
        patho_min = record.get('pathogenic_min')
        age_onset_min = record.get('age_onset_min')
        reference_region = f"{record.get('chrom')}:{record.get('start_hg38')}-{record.get('stop_hg38')}"
        
        if patho_min and normal_max:
            new_record = {
                "LocusId": gene,
                "InheritanceMode": record.get("Inheritance"),
                "DisplayRU": record.get("pathogenic_motif_gene_orientation"),
                "SourceDisplay": record.get("source"),
                #"SourceId": record.get("GeneReviews"),
                "LocusStructure": record.get("locus_structure"),
                "ReferenceRegion":reference_region,
                "VariantType": "Repeat",
                "Disease": re.sub("\/", " ", record.get("disease")), # get rid of forward slashes; causes errors in EPI2ME 
                "NormalMax": int(record.get("normal_max")), # convert floats to ints
                "PathologicMin": int(record.get("pathogenic_min"))        #convert floats to ints
            }

        if patho_min and normal_max and int(age_onset_min)<=age_onset_max:  # make sure critical info is present & filter
            variant_catalog.append(new_record)
            out_straglr_bed.write(f"{record.get('chrom')}\t{record.get('start_hg38')}\t{record.get('stop_hg38')}\t{record.get('reference_motif_reference_orientation')}\t{gene}\t{gene}\n")
            out_nano_bed.write(f"{record.get('chrom')}\t{record.get('start_hg38')}\t{record.get('stop_hg38')}\t{record.get('reference_motif_reference_orientation')}\n")
            #with open('updated_variant_catalog_hg38.json', "a") as file:
            #    json.dump(new_record, file, indent=4)
            # Save the new variant_catalog_hg38.json
            
with open(out_straglr_json_fname, 'w') as file:
    json.dump(variant_catalog, file, indent=4)